# NOTEBOOK 01: KHÁM PHÁ VÀ LÀM SẠCH DỮ LIỆU

## Mục tiêu:
- Tải và khám phá cấu trúc dữ liệu
- Phân tích các giá trị thiếu và ngoại lai
- Làm sạch và tiền xử lý dữ liệu
- Chuẩn bị dữ liệu cho các bước phân tích tiếp theo

## 1. IMPORT CÁC THƯ VIỆN CẦN THIẾT

In [ ]:
# Import các thư viện cơ bản
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats

# Cấu hình hiển thị
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Đã import thành công các thư viện cần thiết!")

## 2. TẢI VÀ KHÁM PHÁ DỮ LIỆU

In [ ]:
# Tải dữ liệu
duongDanFile = '../healthcare-dataset-stroke-data.csv.xls'
duLieuGoc = pd.read_csv(duongDanFile)

print(f"📊 Kích thước dữ liệu: {duLieuGoc.shape}")
print(f"📋 Số dòng: {duLieuGoc.shape[0]:,}")
print(f"📋 Số cột: {duLieuGoc.shape[1]}")
print("\n" + "="*50)
print("🔍 THÔNG TIN TỔNG QUAN VỀ DỮ LIỆU")
print("="*50)
duLieuGoc.info()

In [ ]:
# Hiển thị 5 dòng đầu tiên
print("🔍 5 DÒNG ĐẦU TIÊN CỦA DỮ LIỆU:")
print("="*50)
duLieuGoc.head()

In [ ]:
# Thống kê mô tả cơ bản
print("📈 THỐNG KÊ MÔ TẢ CÁC BIẾN SỐ:")
print("="*50)
duLieuGoc.describe()

In [ ]:
# Thống kê các biến phân loại
print("📊 THỐNG KÊ CÁC BIẾN PHÂN LOẠI:")
print("="*50)

cacBienPhanLoai = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'stroke']

for bien in cacBienPhanLoai:
    print(f"\n🏷️ Biến '{bien}':")
    print("-" * 30)
    giaTriDem = duLieuGoc[bien].value_counts()
    tyLePhanTram = duLieuGoc[bien].value_counts(normalize=True) * 100
    
    ketQua = pd.DataFrame({
        'soLuong': giaTriDem,
        'tyLePhanTram': tyLePhanTram.round(2)
    })
    print(ketQua)

## 3. PHÂN TÍCH GIÁ TRỊ THIẾU

In [ ]:
# Kiểm tra giá trị thiếu
print("🔍 PHÂN TÍCH GIÁ TRỊ THIẾU:")
print("="*50)

giaTriThieu = duLieuGoc.isnull().sum()
tyLeThieu = (duLieuGoc.isnull().sum() / len(duLieuGoc)) * 100

bangGiaTriThieu = pd.DataFrame({
    'soLuongThieu': giaTriThieu,
    'tyLeThieu': tyLeThieu.round(2)
})

bangGiaTriThieu = bangGiaTriThieu[bangGiaTriThieu['soLuongThieu'] > 0].sort_values('soLuongThieu', ascending=False)

if len(bangGiaTriThieu) > 0:
    print(bangGiaTriThieu)
else:
    print("✅ Không có giá trị thiếu trong dữ liệu!")

In [ ]:
# Kiểm tra giá trị 'N/A' trong cột BMI
print("🔍 KIỂM TRA GIÁ TRỊ 'N/A' TRONG CỘT BMI:")
print("="*50)

soLuongNA = (duLieuGoc['bmi'] == 'N/A').sum()
tyLeNA = (soLuongNA / len(duLieuGoc)) * 100

print(f"Số lượng giá trị 'N/A' trong BMI: {soLuongNA}")
print(f"Tỷ lệ giá trị 'N/A': {tyLeNA:.2f}%")

# Hiển thị một số mẫu có BMI = 'N/A'
if soLuongNA > 0:
    print("\n📋 Một số mẫu có BMI = 'N/A':")
    print(duLieuGoc[duLieuGoc['bmi'] == 'N/A'].head())

## 4. LÀM SẠCH VÀ TIỀN XỬ LÝ DỮ LIỆU

In [ ]:
# Tạo bản sao để xử lý
duLieuSach = duLieuGoc.copy()

print("🧹 BẮT ĐẦU QUÁ TRÌNH LÀM SẠCH DỮ LIỆU")
print("="*50)

# 1. Xử lý cột BMI
print("\n1️⃣ Xử lý cột BMI:")
print("-" * 20)

# Chuyển 'N/A' thành NaN
duLieuSach['bmi'] = duLieuSach['bmi'].replace('N/A', np.nan)

# Chuyển đổi kiểu dữ liệu sang float
duLieuSach['bmi'] = pd.to_numeric(duLieuSach['bmi'], errors='coerce')

soLuongNaN = duLieuSach['bmi'].isnull().sum()
print(f"Số lượng giá trị NaN trong BMI sau xử lý: {soLuongNaN}")

# Thay thế giá trị thiếu bằng median
medianBMI = duLieuSach['bmi'].median()
duLieuSach['bmi'].fillna(medianBMI, inplace=True)
print(f"Đã thay thế giá trị thiếu bằng median: {medianBMI:.2f}")

print(f"✅ Hoàn thành xử lý cột BMI")

In [ ]:
# 2. Kiểm tra và xử lý dữ liệu ngoại lai
print("\n2️⃣ Phân tích dữ liệu ngoại lai:")
print("-" * 30)

cacBienSo = ['age', 'avg_glucose_level', 'bmi']

for bien in cacBienSo:
    print(f"\n🔍 Biến '{bien}':")
    
    # Tính toán IQR
    Q1 = duLieuSach[bien].quantile(0.25)
    Q3 = duLieuSach[bien].quantile(0.75)
    IQR = Q3 - Q1
    
    # Xác định ngưỡng ngoại lai
    nguongDuoi = Q1 - 1.5 * IQR
    nguongTren = Q3 + 1.5 * IQR
    
    # Đếm số lượng ngoại lai
    ngoaiLai = duLieuSach[(duLieuSach[bien] < nguongDuoi) | (duLieuSach[bien] > nguongTren)]
    soLuongNgoaiLai = len(ngoaiLai)
    tyLeNgoaiLai = (soLuongNgoaiLai / len(duLieuSach)) * 100
    
    print(f"  - Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
    print(f"  - Ngưỡng: [{nguongDuoi:.2f}, {nguongTren:.2f}]")
    print(f"  - Số lượng ngoại lai: {soLuongNgoaiLai} ({tyLeNgoaiLai:.2f}%)")
    
    if soLuongNgoaiLai > 0:
        print(f"  - Giá trị min ngoại lai: {ngoaiLai[bien].min():.2f}")
        print(f"  - Giá trị max ngoại lai: {ngoaiLai[bien].max():.2f}")

In [ ]:
# 3. Tạo biểu đồ boxplot để trực quan hóa ngoại lai
print("\n📊 TRỰC QUAN HÓA DỮ LIỆU NGOẠI LAI:")
print("="*50)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Biểu đồ Boxplot - Phát hiện dữ liệu ngoại lai', fontsize=16, fontweight='bold')

for i, bien in enumerate(cacBienSo):
    axes[i].boxplot(duLieuSach[bien].dropna())
    axes[i].set_title(f'{bien.upper()}', fontweight='bold')
    axes[i].set_ylabel('Giá trị')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 4. Kiểm tra tính hợp lý của dữ liệu
print("\n3️⃣ Kiểm tra tính hợp lý của dữ liệu:")
print("-" * 40)

# Kiểm tra tuổi
tuoiKhongHopLy = duLieuSach[(duLieuSach['age'] < 0) | (duLieuSach['age'] > 120)]
print(f"Số mẫu có tuổi không hợp lý (< 0 hoặc > 120): {len(tuoiKhongHopLy)}")

# Kiểm tra BMI
bmiKhongHopLy = duLieuSach[(duLieuSach['bmi'] < 10) | (duLieuSach['bmi'] > 60)]
print(f"Số mẫu có BMI không hợp lý (< 10 hoặc > 60): {len(bmiKhongHopLy)}")

# Kiểm tra glucose
glucoseKhongHopLy = duLieuSach[(duLieuSach['avg_glucose_level'] < 50) | (duLieuSach['avg_glucose_level'] > 500)]
print(f"Số mẫu có glucose không hợp lý (< 50 hoặc > 500): {len(glucoseKhongHopLy)}")

# Kiểm tra logic nghiệp vụ
treEmLamViec = duLieuSach[(duLieuSach['age'] < 16) & (duLieuSach['work_type'].isin(['Private', 'Self-employed', 'Govt_job']))]
print(f"Số trẻ em (< 16 tuổi) có công việc người lớn: {len(treEmLamViec)}")

treEmKetHon = duLieuSach[(duLieuSach['age'] < 16) & (duLieuSach['ever_married'] == 'Yes')]
print(f"Số trẻ em (< 16 tuổi) đã kết hôn: {len(treEmKetHon)}")

## 5. TẠO CÁC BIẾN MỚI VÀ MÃ HÓA DỮ LIỆU

In [ ]:
# Tạo các biến mới
print("🔧 TẠO CÁC BIẾN MỚI:")
print("="*50)

# 1. Phân loại tuổi
def phanLoaiTuoi(tuoi):
    if tuoi < 18:
        return 'treEm'
    elif tuoi < 30:
        return 'tuoiTre'
    elif tuoi < 50:
        return 'trungNien'
    elif tuoi < 65:
        return 'truocTuoiHuu'
    else:
        return 'tuoiCao'

duLieuSach['nhomTuoi'] = duLieuSach['age'].apply(phanLoaiTuoi)
print("✅ Đã tạo biến 'nhomTuoi'")

# 2. Phân loại BMI
def phanLoaiBMI(bmi):
    if bmi < 18.5:
        return 'thieuCan'
    elif bmi < 25:
        return 'binhThuong'
    elif bmi < 30:
        return 'thua Can'
    else:
        return 'beoPhi'

duLieuSach['nhomBMI'] = duLieuSach['bmi'].apply(phanLoaiBMI)
print("✅ Đã tạo biến 'nhomBMI'")

# 3. Phân loại glucose
def phanLoaiGlucose(glucose):
    if glucose < 100:
        return 'binhThuong'
    elif glucose < 126:
        return 'tienTieuDuong'
    else:
        return 'tieuDuong'

duLieuSach['nhomGlucose'] = duLieuSach['avg_glucose_level'].apply(phanLoaiGlucose)
print("✅ Đã tạo biến 'nhomGlucose'")

# 4. Tạo biến tổng hợp nguy cơ
duLieuSach['diemNguyCo'] = (
    duLieuSach['hypertension'] + 
    duLieuSach['heart_disease'] + 
    (duLieuSach['age'] > 65).astype(int) +
    (duLieuSach['bmi'] > 30).astype(int) +
    (duLieuSach['avg_glucose_level'] > 126).astype(int)
)
print("✅ Đã tạo biến 'diemNguyCo'")

In [ ]:
# Hiển thị thống kê các biến mới
print("\n📊 THỐNG KÊ CÁC BIẾN MỚI:")
print("="*50)

cacBienMoi = ['nhomTuoi', 'nhomBMI', 'nhomGlucose']

for bien in cacBienMoi:
    print(f"\n🏷️ Biến '{bien}':")
    print("-" * 30)
    giaTriDem = duLieuSach[bien].value_counts()
    tyLePhanTram = duLieuSach[bien].value_counts(normalize=True) * 100
    
    ketQua = pd.DataFrame({
        'soLuong': giaTriDem,
        'tyLePhanTram': tyLePhanTram.round(2)
    })
    print(ketQua)

print(f"\n🏷️ Biến 'diemNguyCo':")
print("-" * 30)
print(duLieuSach['diemNguyCo'].describe())

## 6. KIỂM TRA CHẤT LƯỢNG DỮ LIỆU SAU XỬ LÝ

In [ ]:
# Kiểm tra chất lượng dữ liệu cuối cùng
print("✅ KIỂM TRA CHẤT LƯỢNG DỮ LIỆU SAU XỬ LÝ:")
print("="*50)

print(f"📊 Kích thước dữ liệu sau xử lý: {duLieuSach.shape}")
print(f"📋 Số dòng: {duLieuSach.shape[0]:,}")
print(f"📋 Số cột: {duLieuSach.shape[1]}")

# Kiểm tra giá trị thiếu
giaTriThieuSauXuLy = duLieuSach.isnull().sum().sum()
print(f"\n🔍 Tổng số giá trị thiếu: {giaTriThieuSauXuLy}")

# Kiểm tra trùng lặp
soLuongTrungLap = duLieuSach.duplicated().sum()
print(f"🔍 Số dòng trùng lặp: {soLuongTrungLap}")

# Thống kê biến mục tiêu
print(f"\n🎯 Phân phối biến mục tiêu 'stroke':")
phanPhoiStroke = duLieuSach['stroke'].value_counts()
tyLePhanPhoiStroke = duLieuSach['stroke'].value_counts(normalize=True) * 100

bangPhanPhoi = pd.DataFrame({
    'soLuong': phanPhoiStroke,
    'tyLePhanTram': tyLePhanPhoiStroke.round(2)
})
bangPhanPhoi.index = ['Không đột quỵ', 'Có đột quỵ']
print(bangPhanPhoi)

## 7. LUU DỮ LIỆU ĐÃ XỬ LÝ

In [ ]:
# Lưu dữ liệu đã xử lý
duongDanLuu = '../data_processed/du_lieu_da_xu_ly.csv'

# Tạo thư mục nếu chưa tồn tại
import os
os.makedirs('../data_processed', exist_ok=True)

# Lưu file
duLieuSach.to_csv(duongDanLuu, index=False)
print(f"💾 Đã lưu dữ liệu đã xử lý vào: {duongDanLuu}")

# Hiển thị thông tin tóm tắt
print("\n" + "="*60)
print("📋 TÓM TẮT QUÁ TRÌNH XỬ LÝ DỮ LIỆU")
print("="*60)
print(f"✅ Dữ liệu gốc: {duLieuGoc.shape[0]:,} dòng, {duLieuGoc.shape[1]} cột")
print(f"✅ Dữ liệu sau xử lý: {duLieuSach.shape[0]:,} dòng, {duLieuSach.shape[1]} cột")
print(f"✅ Đã xử lý {soLuongNA} giá trị thiếu trong cột BMI")
print(f"✅ Đã tạo {len(cacBienMoi) + 1} biến mới")
print(f"✅ Dữ liệu đã sẵn sàng cho các bước phân tích tiếp theo")
print("="*60)